In [1]:
import pandas as pd
import numpy as np
import implicit as im
from scipy.sparse import coo_matrix
from sklearn import preprocessing

In [2]:
df=pd.read_csv('dataset_1.csv')
df=df.drop(['色相','Unnamed: 0','品名','色调','款式流水号'],axis=1)
print(df.columns)
from sklearn.model_selection import train_test_split
X_train, X_test,= train_test_split(df, test_size = 0.3)

Index(['客户ID', '衣服单价', '衣服ID', '颜色深度', '年份', '季节', '小类号码', '颜色式样', '颜色大类',
       '颜色名称', '大类', '中类', '品牌定义廓形', '领型', '袖长', '系列', '尺寸组', '标准价', '色温',
       '色明度', '色纯度', '对比度', '廓形', '宽松度', '长度', '款设图-大小复杂度', '款设图-规则性',
       '款设图-曲直', '材质-垂感', '整体曲直', '量感', '场合', '领型-形状', '领型-曲直', '领型-量感',
       '裙型裤型'],
      dtype='object')


In [3]:
def read_data_latent(filename):
    """ Reads in the last.fm dataset, and returns a tuple of a pandas dataframe
    and a sparse matrix of artist/user/playcount """
    # read in triples of user/artist/playcount from the input dataset
    # get a model based off the input params
    start = time.time()
    logging.debug("reading data from %s", filename)
    data = pd.read_csv(filename,
                             usecols=[0, 1, 2],
                             names=['user', 'item', 'buys'],
                             na_filter=False)
    # map each artist and user to a unique numeric value
    data['user'] = data['user'].astype("category")#降低内存使用不会改变显示,将object对象映射为int
    data['item'] = data['item'].astype("category")
    
    # create a sparse matrix of all the users/plays
    buys = coo_matrix((data['buys'].astype(np.float32),
                       (data['item'].cat.codes.copy(),#Series.cat.codes 属性来返回 category 类型用来表示每个值的整型值
                        data['user'].cat.codes.copy())))
    
    logging.debug("read data file in %s", time.time() - start)
    return data, buys

In [4]:
raw=X_train
#请改成train_set

raw["buys"]=1
raw.groupby(["客户ID","衣服ID"])["buys"].sum().to_csv('uib.csv')
hot_items=X_train["衣服ID"].value_counts()
hot_items=hot_items.index
#生成用户购买表

from implicit.als import AlternatingLeastSquares as als
import time
import logging
model=als(factors=40, dtype= np.float32, use_gpu= False)
#调用als模型
data,buys=read_data_latent('uib.csv')
logging.debug("training model %s", "als")
buys = buys.tocsr()
start = time.time()
model.fit(buys)
logging.debug("trained model '%s' in %0.2fs", "als", time.time() - start)


T='JWWC41201026'
item_factors=model.item_factors

#生成Item顺序和衣服ID的映射关系

artists = dict(enumerate(data['item'].cat.categories))
artist={artists[i]:i for i in artists}


D:\anaconda\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [5]:
buys.shape

(285, 143)

In [6]:
item_factors

array([[ 0.02173986,  0.02166602,  0.00439817, ...,  0.02700594,
        -0.02245905,  0.00270304],
       [ 0.00530102, -0.00052387,  0.04038437, ...,  0.10605805,
        -0.08482912, -0.01631423],
       [-0.09432297,  0.06441087,  0.0265511 , ...,  0.06031375,
        -0.02009629,  0.02895735],
       ...,
       [-0.00934038,  0.04282898, -0.04337749, ..., -0.02423061,
        -0.05753578, -0.0550958 ],
       [-0.00703733,  0.01125997,  0.02780131, ...,  0.05502462,
        -0.0152658 , -0.04656406],
       [-0.07128552, -0.08511844,  0.01543678, ..., -0.01221802,
         0.10437804,  0.00292434]], dtype=float32)

In [7]:
itemID1=[];
for i in range(item_factors.shape[0]):
    itemID1.append(artists[i])
len(itemID1)

285

In [8]:
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
#item_factors = min_max_scaler.fit_transform(item_factors)
#item_factors = preprocessing.normalize(item_factors, norm='l2')#L2正则化
pd_item_factor=pd.DataFrame(item_factors,index=itemID1)

In [9]:
item_features=pd.read_csv('item_feature.csv')
item_features.index=item_features['衣服ID']
item_features=item_features.drop(['衣服ID'],axis=1)

In [10]:
a=1
b=0.2
fillnum=0.1

In [11]:
itemID=item_features.index
#item_features = min_max_scaler.fit_transform(item_features)
#item_features=preprocessing.normalize(item_features, norm='l2')
item_features =pd.DataFrame(item_features,index=itemID)
item_features=pd.merge(item_features*a,pd_item_factor*b, how='left',right_index=True,left_index=True)
item_features=item_features.fillna(fillnum)

In [12]:
itemID=item_features.index
item_matrix=item_features.as_matrix(columns=None)
item_ID={}
for i in range(len(itemID)):
    item_ID[itemID[i]]=i

In [13]:
userset=list(set(data['user']))

user_ID={}
for i in range(len(userset)):
    user_ID[userset[i]]=i
user_profile=np.zeros((len(userset),item_features.shape[1]))

for user,itemid,buys in data.values:  #计算user_profile  
    user_profile[user_ID[user]]+=item_matrix[item_ID[itemid]]*buys

In [14]:
user_profile =preprocessing.normalize(user_profile, norm='l2')#正则化

In [15]:
print(item_matrix.shape)
print(user_profile.shape)

(322, 337)
(143, 337)


In [16]:
scores=user_profile.dot(item_matrix.T)

In [17]:
usr_train=X_train.groupby("客户ID")
usr_test=X_test.groupby("客户ID")
def recoder(od_dataset):
    usr_item_dict={usr:[i for i in groups['衣服ID']]for usr, groups in list(od_dataset)}
    return usr_item_dict
usr_item_dict_train=recoder(usr_train)
usr_item_dict_test=recoder(usr_test)
itemid={item_ID[i]:i for i in item_ID}
def rank(array,item):
    x=pd.Series(array)    
    y=pd.Series(item)
    y=y.map(x)
    y=dict(y)
    return y    

In [18]:
recommender={}
for i in user_ID:
    x=rank(scores[user_ID[i]],item_ID)    
    for item in usr_item_dict_train[i]:
        try:
            x.pop(item)
        except:
            pass                
    recommender[i]=x
for d in recommender:
    a=recommender[d]
    recommender[d]=sorted(a.items(),key=lambda a:a[1],reverse=True) 

In [19]:
hot_items=list(hot_items)

In [20]:
b=X_test.groupby('客户ID')
sum=0
fenzi=0
for user,grouped in b:
    num=len(grouped['衣服ID'])
    #分母
    if user not in recommender:
        for item in grouped['衣服ID']:
            for item in grouped['衣服ID']: 
                try:                    
                    rank=(hot_items.index(item)+1)/len(hot_items)
                    fenzi+=rank
                except:
                    num-=1
    else:        
        g=recommender[user]
        g=[i for (i,j) in g]
        for item in grouped['衣服ID']: 
            try:
                rank=(g.index(item)+1)/len(g)
                fenzi+=rank
            except:                
                num-=1
    sum+=num
rank_1=fenzi/sum
print(rank_1)

0.425511733360827


In [21]:
#import pymysql

# 打开数据库连接
db =pymysql.connect(host='localhost', port=3306, user='root', passwd='root', db='weiyitech')                     
cursor = db.cursor()
for user in recommender:
    print(type(user))
    x=recommender[user];
    g=[i for (i,j) in x]
    print(g[0])
    # SQL 插入语句
       # 执行sql语句
    cursor.execute("INSERT INTO user_recommend(user_id, item1, item2, item3, item4,item5,item6,item7,item8,item9,item10) VALUES\
    (%s, %s, %s, %s, %s, %s, %s, %s,  %s, %s, %s )",[user,g[0],g[1],g[2],g[3],g[4],g[5],g[6],g[7],g[8],g[9]])
       # 提交到数据库执行
    db.commit()

 # 关闭数据库连接
db.close()

NameError: name 'pymysql' is not defined